In [42]:
import requests
import pandas as pd
import numpy as np

import folium

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

from pandas.io.json import json_normalize





In [2]:
#Define Four Square credentials and version

CLIENT_ID = 'XXXX' # your Foursquare ID
CLIENT_SECRET = 'XXXX' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 2G4X5FWIUCO43DTFHL5MC1AQWXGKEKSF10RBYBI5JLETLHRW
CLIENT_SECRET:N0X2UYWNML2O2R10DCOAO3XYPJBIN14PCOBBWWPNSYM3V4DX


# Bangalore City Exploration

Read the Bangalore city neighborhoods from the csv file

In [3]:
# Bangalore Suburbs Details

blr = pd.read_csv("Bangalore_Suburb_Names.csv")
blr.head()

# Add Bangalore name in the behind

blr['Neighborhood'] = blr['Neighborhood'].astype(str).replace('_', '') + " Bangalore"

Blr_list = list(blr['Neighborhood'])

Blr_list


['Agara Bangalore',
 'Arekere Bangalore',
 'Banashankari Bangalore',
 'Banaswadi Bangalore',
 'Basavanagudi Bangalore',
 'Begur Bangalore',
 'Bellandur Bangalore',
 'BEML Bangalore',
 'Bengaluru Pete Bangalore',
 'Bidadi Bangalore',
 'Bommasandra Bangalore',
 'Brigade Road Bangalore',
 'Chandapura Bangalore',
 'Devanahalli Bangalore',
 'Dhobi Ghat Bangalore',
 'Domlur Bangalore',
 'Electronics City Bangalore',
 'Fraser Town Bangalore',
 'Girinagar Bangalore',
 'HBR Layout Bangalore',
 'Hebbal Bangalore',
 'Hosakote Bangalore',
 'HSR Layout Bangalore',
 'Indiranagar Bangalore',
 'Ittamadu Bangalore',
 'Ittamadu Bangalore',
 'J. P. Nagar Bangalore',
 'Jakkur Bangalore',
 'Jayanagar Bangalore',
 'Jeevanbheemanagar Bangalore',
 'Kettohalli Bangalore',
 'Kodihalli Bangalore',
 'Konanakunte Bangalore',
 'Koramangala Bangalore',
 'Krishnarajapura Bangalore',
 'Kundalahalli Bangalore',
 'Madiwala Bangalore',
 'Magadi Bangalore',
 'Mahadevapura Bangalore',
 'Majestic Bangalore',
 'Malleswaram B

Calculate the coordinates using the GeoCoder

In [7]:
#Calculate the coordinates

neigh = []
lat = []
lon = []

from geopy.exc import GeocoderTimedOut

for i in Blr_list:
    
    try:
        geolocator = Nominatim(user_agent="foursquare_agent")
        location = geolocator.geocode(i)
        latitude = location.latitude
        longitude = location.longitude
        print(i, latitude, longitude)
        neigh.append(i)
        lat.append(latitude)
        lon.append(longitude)
        
    except AttributeError as error:
        pass

    except NameError as error:
        continue
    
    except GeocoderTimedOut as error:
        continue
    



Agara Bangalore 12.6201123 77.4793074
Arekere Bangalore 12.8872086 77.5960493
Banashankari Bangalore 12.9152208 77.573598
Banaswadi Bangalore 13.0141618 77.6518539
Basavanagudi Bangalore 12.9417261 77.5755021
Begur Bangalore 12.8633887 77.61301120503904
Bellandur Bangalore 12.9791198 77.5912997
BEML Bangalore 12.98673235 77.65219444193485
Bidadi Bangalore 12.7988507 77.3869947
Brigade Road Bangalore 12.9750781 77.6079683
Chandapura Bangalore 12.8004439 77.7062826
Dhobi Ghat Bangalore 12.98081635 77.62274235223254
Domlur Bangalore 12.9624669 77.6381958
Electronics City Bangalore 12.8468545 77.6769267
Girinagar Bangalore 12.9401439 77.5444995
HBR Layout Bangalore 13.0358698 77.6323597
Hebbal Bangalore 13.0382184 77.5919
Hosakote Bangalore 13.4524681 77.4762302
HSR Layout Bangalore 12.9116225 77.6388622
Indiranagar Bangalore 12.9732913 77.6404672
Ittamadu Bangalore 12.9250047 77.5444573
J. P. Nagar Bangalore 12.909492199999999 77.59673855731768
Jayanagar Bangalore 12.9292731 77.5824229
Ko

In [8]:
# Creates the dataframe
df = pd.DataFrame()

df['Neighborhood'] = neigh
df['Latitude'] = lat
df['Longitude'] = lon

df.to_csv('Bangalore_Suburb_Names_Coordinates')
df.head()

,Neighborhood,Latitude,Longitude
0,Agara Bangalore,12.620112,77.479307
1,Arekere Bangalore,12.887209,77.596049
2,Banashankari Bangalore,12.915221,77.573598
3,Banaswadi Bangalore,13.014162,77.651854
4,Basavanagudi Bangalore,12.941726,77.575502


In [179]:
import folium


# create map of Bangaloreusing latitude and longitude values
map_Bangalore = folium.Map(location=[12.9716, 77.5946], zoom_start=10)

# add markers to map
for lat, lng, neighborhood in zip(df['Latitude'], df['Longitude'], df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.5,
        parse_html=False).add_to(map_Bangalore)  
    
map_Bangalore

# Neighborhood Exploration using FourSquare API

In [106]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['location']['distance'],
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Distance',           
                  'Venue Category']
    
    return(nearby_venues)

In [107]:
#Use above function to pull venues near neighborhood
LIMIT = 500

Bangalore_venues = getNearbyVenues(names = df['Neighborhood'], latitudes = df['Latitude'], 
                                longitudes = df['Longitude'])

Agara Bangalore
Arekere Bangalore
Banashankari Bangalore
Banaswadi Bangalore
Basavanagudi Bangalore
Begur Bangalore
Bellandur Bangalore
BEML Bangalore
Bidadi Bangalore
Brigade Road Bangalore
Chandapura Bangalore
Dhobi Ghat Bangalore
Domlur Bangalore
Electronics City Bangalore
Girinagar Bangalore
HBR Layout Bangalore
Hebbal Bangalore
Hosakote Bangalore
HSR Layout Bangalore
Indiranagar Bangalore
Ittamadu Bangalore
J. P. Nagar Bangalore
Jayanagar Bangalore
Kodihalli Bangalore
Konanakunte Bangalore
Koramangala Bangalore
Kundalahalli Bangalore
Magadi Bangalore
Mahadevapura Bangalore
Majestic Bangalore
Malleswaram Bangalore
Mathikere Bangalore
Murugeshpalya Bangalore
Nagarbhavi Bangalore
Rajajinagar Bangalore
Rajarajeshwari Nagar Bangalore
Ramamurthy Nagar Bangalore
Sahakara Nagar Bangalore
Seetharampalya Bangalore
Thubarahalli Bangalore
Tippasandra Bangalore
UB City Bangalore
Varthur Bangalore
Yelahanka Bangalore
Yeshwantpur Bangalore


In [123]:
Bangalore_venues

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Distance,Venue Category
0,Arekere Bangalore,12.887209,77.596049,Decathlon Sports India Pvt Ltd,12.887513,77.597712,183,Sporting Goods Shop
1,Arekere Bangalore,12.887209,77.596049,Natural Ice Cream,12.892188,77.598222,602,Ice Cream Shop
2,Arekere Bangalore,12.887209,77.596049,Chavadi,12.892199,77.602538,896,BBQ Joint
3,Arekere Bangalore,12.887209,77.596049,Guru Garden,12.891196,77.597979,490,Indian Restaurant
4,Arekere Bangalore,12.887209,77.596049,Zhangs Dynasty,12.889923,77.597804,357,Chinese Restaurant
...,...,...,...,...,...,...,...,...
1333,Yeshwantpur Bangalore,13.023830,77.552921,Vivantha By Taj,13.022276,77.550766,290,Hotel
1334,Yeshwantpur Bangalore,13.023830,77.552921,Vaishnavi Sapphire Mall,13.024272,77.548644,466,Shopping Mall
1335,Yeshwantpur Bangalore,13.023830,77.552921,Thate Idli Point,13.025136,77.558553,627,Breakfast Spot
1336,Yeshwantpur Bangalore,13.023830,77.552921,United Colors of Benetton,13.027116,77.558290,687,Clothing Store


In [112]:
Bangalore_venues[Bangalore_venues['Neighborhood'] == 'Majestic Bangalore'].head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Distance,Venue Category
910,Majestic Bangalore,12.975708,77.572876,Hotel Fishland,12.975569,77.578592,620,Seafood Restaurant
911,Majestic Bangalore,12.975708,77.572876,Foto Circle,12.980221,77.574731,541,Electronics Store
912,Majestic Bangalore,12.975708,77.572876,Udupi Sri Krishna Bhavan,12.971563,77.574158,481,Indian Restaurant
913,Majestic Bangalore,12.975708,77.572876,SGS Donne Biriyani,12.970325,77.572648,599,Indian Restaurant
914,Majestic Bangalore,12.975708,77.572876,Sangam Sweets,12.976924,77.577891,560,Dessert Shop


In [120]:
Bangalore_venues[Bangalore_venues['Neighborhood'] == 'Majestic Bangalore'].groupby('Venue Category').count()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Distance
Venue Category,,,,,,,
Bed & Breakfast,2,2,2,2,2,2,2
Bookstore,1,1,1,1,1,1,1
Bus Station,1,1,1,1,1,1,1
Café,1,1,1,1,1,1,1
Dessert Shop,1,1,1,1,1,1,1
Electronics Store,1,1,1,1,1,1,1
Fast Food Restaurant,1,1,1,1,1,1,1
Flea Market,1,1,1,1,1,1,1
Hotel,3,3,3,3,3,3,3


In [132]:
Bangalore_venues.groupby('Venue Category').count().sort_values('Neighborhood', ascending = False).head(10)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Distance
Venue Category,,,,,,,
Indian Restaurant,198,198,198,198,198,198,198
Café,92,92,92,92,92,92,92
Hotel,45,45,45,45,45,45,45
Coffee Shop,41,41,41,41,41,41,41
Ice Cream Shop,36,36,36,36,36,36,36
Fast Food Restaurant,34,34,34,34,34,34,34
Pizza Place,33,33,33,33,33,33,33
Chinese Restaurant,32,32,32,32,32,32,32
Lounge,29,29,29,29,29,29,29


In [133]:
Bangalore_venues.columns

Index(['Neighborhood', 'Neighborhood Latitude', 'Neighborhood Longitude',
       'Venue', 'Venue Latitude', 'Venue Longitude', 'Venue Distance',
       'Venue Category'],
      dtype='object')

In [157]:
Bangalore_venues.groupby('Neighborhood').count().sort_values('Neighborhood Latitude', ascending = False)

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Distance,Venue Category
Neighborhood,,,,,,,
Jayanagar Bangalore,100,100,100,100,100,100,100
UB City Bangalore,100,100,100,100,100,100,100
Brigade Road Bangalore,100,100,100,100,100,100,100
Indiranagar Bangalore,100,100,100,100,100,100,100
Domlur Bangalore,100,100,100,100,100,100,100
Bellandur Bangalore,71,71,71,71,71,71,71
HSR Layout Bangalore,64,64,64,64,64,64,64
Dhobi Ghat Bangalore,63,63,63,63,63,63,63
Malleswaram Bangalore,61,61,61,61,61,61,61


In [148]:
Bangalore_venues['Venue Category'].unique()

array(['Sporting Goods Shop', 'Ice Cream Shop', 'BBQ Joint',
       'Indian Restaurant', 'Chinese Restaurant', 'Liquor Store',
       'Pizza Place', 'Fast Food Restaurant', 'Department Store',
       'Burger Joint', 'Café', 'Athletics & Sports',
       'Gym / Fitness Center', 'Bar', 'Badminton Court', 'Bakery',
       'Coffee Shop', 'South Indian Restaurant',
       'Mediterranean Restaurant', 'Fish Market', 'Boutique',
       'Breakfast Spot', "Women's Store", 'Seafood Restaurant',
       'Hookah Bar', 'Jewelry Store', 'Karnataka Restaurant',
       'Metro Station', 'Diner', 'Restaurant', 'Park',
       'Vegetarian / Vegan Restaurant', 'Arts & Crafts Store',
       'Andhra Restaurant', 'Bistro', 'Kerala Restaurant', 'Bus Station',
       'Music Venue', 'Dessert Shop', 'Sandwich Place', 'Food Truck',
       'Snack Place', 'Farmers Market', 'Asian Restaurant', 'Tea Room',
       'Juice Bar', 'Hotel', 'Plaza', 'Italian Restaurant', 'Lake',
       'Fruit & Vegetable Store', 'Playground', 

In [149]:
# one hot encoding

Bangalore_onehot = pd.get_dummies(Bangalore_venues[['Venue Category']], prefix = "", prefix_sep = "")

# add Neighbour hood column
Bangalore_onehot['Neighbourhood'] = Bangalore_venues['Neighborhood']

# move the last column to first
fixed_col = [Bangalore_onehot.columns[-1]] + list(Bangalore_onehot.columns[:-1])
Bangalore_onehot = Bangalore_onehot[fixed_col]

Bangalore_onehot

,Neighbourhood,ATM,Accessories Store,Afghan Restaurant,American Restaurant,Andhra Restaurant,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,...,Theme Park,Toy / Game Store,Track Stadium,Trail,Train Station,Udupi Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Women's Store
0,Arekere Bangalore,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Arekere Bangalore,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Arekere Bangalore,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Arekere Bangalore,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Arekere Bangalore,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1333,Yeshwantpur Bangalore,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1334,Yeshwantpur Bangalore,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1335,Yeshwantpur Bangalore,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1336,Yeshwantpur Bangalore,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [150]:
Bangalore_grouped = Bangalore_onehot.groupby('Neighbourhood').mean().reset_index()
Bangalore_grouped

,Neighbourhood,ATM,Accessories Store,Afghan Restaurant,American Restaurant,Andhra Restaurant,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,...,Theme Park,Toy / Game Store,Track Stadium,Trail,Train Station,Udupi Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Women's Store
0,Arekere Bangalore,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000
1,BEML Bangalore,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000
2,Banashankari Bangalore,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.055556
3,Banaswadi Bangalore,0.0,0.000000,0.00,0.000000,0.052632,0.000000,0.000000,0.052632,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.052632,0.000000,0.00,0.000000
4,Basavanagudi Bangalore,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000
5,Begur Bangalore,0.2,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000
6,Bellandur Bangalore,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.014085,0.014085,0.028169,...,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000
7,Bidadi Bangalore,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.200000,0.00,0.000000,0.000000,0.00,0.000000
8,Brigade Road Bangalore,0.0,0.000000,0.01,0.020000,0.010000,0.000000,0.010000,0.010000,0.000000,...,0.000000,0.020000,0.00,0.000000,0.000000,0.00,0.010000,0.000000,0.00,0.020000
9,Chandapura Bangalore,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000


In [138]:
num_top_venues = 10
for neigh in Bangalore_grouped['Neighbourhood']:
    print("----"+neigh+"----")
    temp = Bangalore_grouped[Bangalore_grouped['Neighbourhood'] == neigh].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Arekere Bangalore----
                  venue  freq
0        Ice Cream Shop  0.10
1           Pizza Place  0.10
2      Department Store  0.10
3     Indian Restaurant  0.06
4                  Café  0.06
5    Chinese Restaurant  0.06
6   Sporting Goods Shop  0.06
7           Fish Market  0.03
8  Fast Food Restaurant  0.03
9          Liquor Store  0.03


----BEML Bangalore----
                            venue  freq
0                    Intersection  0.22
1              Light Rail Station  0.11
2                     Pizza Place  0.11
3                        Platform  0.11
4               Indian Restaurant  0.11
5                     Bus Station  0.11
6                         Butcher  0.11
7                            Café  0.11
8                    Noodle House  0.00
9  Multicuisine Indian Restaurant  0.00


----Banashankari Bangalore----
                  venue  freq
0     Indian Restaurant  0.22
1                  Café  0.11
2         Women's Store  0.06
3        Breakfast Spot  0


----Kodihalli Bangalore----
                            venue  freq
0        Mediterranean Restaurant   1.0
1                             ATM   0.0
2       Middle Eastern Restaurant   0.0
3      Modern European Restaurant   0.0
4                 Motorcycle Shop   0.0
5                   Movie Theater   0.0
6  Multicuisine Indian Restaurant   0.0
7                       Multiplex   0.0
8                     Music Store   0.0
9                     Music Venue   0.0


----Konanakunte Bangalore----
                           venue  freq
0                            ATM   0.2
1  Vegetarian / Vegan Restaurant   0.2
2                         Bakery   0.2
3               Department Store   0.2
4           Gym / Fitness Center   0.2
5                       Pharmacy   0.0
6                      Nightclub   0.0
7     Modern European Restaurant   0.0
8                Motorcycle Shop   0.0
9                  Movie Theater   0.0


----Kundalahalli Bangalore----
                  venue  freq
0      

In [139]:
# Function to Sort the venues in decending order

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [140]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = Bangalore_grouped['Neighbourhood']

for ind in np.arange(Bangalore_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Bangalore_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.shape

(42, 11)

In [158]:
neighborhoods_venues_sorted.head(10)

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Arekere Bangalore,Pizza Place,Department Store,Ice Cream Shop,Sporting Goods Shop,Chinese Restaurant,Café,Indian Restaurant,South Indian Restaurant,Bar,Boutique
1,BEML Bangalore,Intersection,Pizza Place,Bus Station,Light Rail Station,Platform,Café,Butcher,Indian Restaurant,Arts & Crafts Store,French Restaurant
2,Banashankari Bangalore,Indian Restaurant,Café,Women's Store,South Indian Restaurant,Hookah Bar,Jewelry Store,Karnataka Restaurant,Metro Station,Diner,Park
3,Banaswadi Bangalore,Indian Restaurant,Kerala Restaurant,Café,BBQ Joint,Music Venue,Department Store,Bus Station,Seafood Restaurant,Bistro,Pizza Place
4,Basavanagudi Bangalore,Indian Restaurant,Fast Food Restaurant,Ice Cream Shop,Café,Sandwich Place,Hotel,Lake,Coffee Shop,Park,Pizza Place
5,Begur Bangalore,ATM,Playground,Fruit & Vegetable Store,Resort,Lake,Donut Shop,Flea Market,Fish Market,Fast Food Restaurant,Farmers Market
6,Bellandur Bangalore,Indian Restaurant,Hotel,Italian Restaurant,Coffee Shop,Lounge,Mexican Restaurant,Café,Japanese Restaurant,Park,Electronics Store
7,Bidadi Bangalore,Indian Restaurant,Train Station,Breakfast Spot,Creperie,Cricket Ground,Food & Drink Shop,Food,Flower Shop,Flea Market,Fish Market
8,Brigade Road Bangalore,Indian Restaurant,Café,Clothing Store,Hotel,Pub,Ice Cream Shop,Chinese Restaurant,Lounge,Steakhouse,Women's Store
9,Chandapura Bangalore,Indian Restaurant,Restaurant,Café,Motorcycle Shop,Department Store,Bakery,Flea Market,Fish Market,Fast Food Restaurant,Farmers Market


# Clustering - K-Means Algorithm

In [151]:
from sklearn.cluster import KMeans

kclusters = 6 # no of cluster as 6

Bangalore_grouped_clustering = Bangalore_grouped.drop('Neighbourhood', axis = 1)

kmeans = KMeans(n_clusters = kclusters, random_state = 1 ).fit(Bangalore_grouped_clustering)

In [152]:
kmeans.labels_

array([1, 1, 1, 5, 5, 4, 1, 5, 1, 5, 1, 1, 1, 1, 1, 1, 5, 1, 1, 1, 1, 2,
       1, 1, 3, 1, 5, 1, 1, 1, 5, 5, 1, 1, 1, 1, 1, 5, 1, 0, 1, 1],
      dtype=int32)

In [153]:
df_Cluster = pd.DataFrame()
df_Cluster['Neighborhood'] = Bangalore_grouped['Neighbourhood']
df_Cluster['Cluster'] = kmeans.labels_

In [154]:
final = df_Cluster.merge(df, on= 'Neighborhood')
final

,Neighborhood,Cluster,Latitude,Longitude
0,Arekere Bangalore,1,12.887209,77.596049
1,BEML Bangalore,1,12.986732,77.652194
2,Banashankari Bangalore,1,12.915221,77.573598
3,Banaswadi Bangalore,5,13.014162,77.651854
4,Basavanagudi Bangalore,5,12.941726,77.575502
5,Begur Bangalore,4,12.863389,77.613011
6,Bellandur Bangalore,1,12.979120,77.591300
7,Bidadi Bangalore,5,12.798851,77.386995
8,Brigade Road Bangalore,1,12.975078,77.607968
9,Chandapura Bangalore,5,12.800444,77.706283


In [155]:
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map of New York using latitude and longitude values
map_cluster = folium.Map(location=[12.9716, 77.5946], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to map
for lat, lng, neighborhood, cluster in zip(final['Latitude'], final['Longitude'],
                                           final['Neighborhood'], final['Cluster']):
    
    label = folium.Popup( ' Cluster ' + str(cluster) + neighborhood, parse_html = True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color= rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.9,
        parse_html=False).add_to(map_cluster)  
    
map_cluster

# Case Study1: Recommend a Place

In [181]:
list = ['Murugeshpalya Bangalore', 'Tippasandra Bangalore'] 

df1 = df[df['Neighborhood'].isin(list)]
df1



,Neighborhood,Latitude,Longitude
32,Murugeshpalya Bangalore,12.958948,77.655593
40,Tippasandra Bangalore,12.976716,77.658222


In [186]:


import folium


# create map of Bangaloreusing latitude and longitude values
map_Bangalore = folium.Map(location=[12.9716, 77.5946], zoom_start=20)

# add markers to map
for lat, lng, neighborhood in zip(df1['Latitude'], df1['Longitude'], df1['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.5,
        parse_html=True).add_to(map_Bangalore)  
    
map_Bangalore

In [178]:
#Use above function to pull venues near neighborhood
LIMIT = 100
search_query = 'Metro Station'
latitude = 12.976716
longitude = 77.658222 
radius=10000


url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            latitude, 
            longitude, 
            VERSION, 
            search_query, 
            radius, 
            LIMIT)


results = requests.get(url).json()
results

# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]



df_aa = dataframe_filtered[(dataframe_filtered['categories'] == search_query) ]

df_aa[['name','categories','lat','lng','distance']].sort_values('distance')

,name,categories,lat,lng,distance
8,Byappanahalli Metro Station,Metro Station,12.995587,77.658733,2101
2,Indiranagar Metro Station,Metro Station,12.978266,77.638634,2131
7,Trinity Metro Station,Metro Station,12.972948,77.617340,4454
12,MG Road Metro Station,Metro Station,12.975495,77.606777,5582
19,Cubbon Park Metro Station,Metro Station,12.980437,77.597878,6558
20,Shri M Visvesvaraya Metro Station,Metro Station,12.974097,77.584015,8054
13,Chickpet Metro Station,Metro Station,12.966987,77.574840,9109
21,K R Market Metro Station,Metro Station,12.960549,77.574974,9208
17,Kempegowda Metro Station,Metro Station,12.976134,77.571482,9409
24,South End Circle Metro Station,Metro Station,12.938359,77.579884,9510
